#Step 1. Install EleutherAI Evaluations Harness
*   Logging into WandB is optional.
*   Logging into Huggingface API is required to run GPQA. This is to prevent database leakage.
*   Uses Goodfire API! Experimental as of Jan 2025

In [3]:
# Remove Google Colab specific import and install commands
import os
import huggingface_hub

# Install dependencies using pip in a separate cell if needed
!pip install -r requirements.txt

Obtaining lm_eval[vllm,wandb] from git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] (from lm_eval[vllm,wandb])
  Updating c:\users\minh1\local\lm-evaluation-harness\src\lm-eval clone
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached numexpr-2.10

DEPRECATION: git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git fetch -q --tags
  Running command git reset --hard -q 95ce7c50956fc0c1713b1fa7441fdf592b3ae56c
ERROR: Could not find a version that satisfies the requirement torch>=1.8 (from lm-eval) (from versions: none)
ERROR: No matching distribution found for torch>=1.8


Automated login for Hugging Face Hub via Colab Secrets. If you don't have this, it'll prompt for manual login if you don't have one. If you completely remove this, you can't run GPQA or use Llama models via HF.

In [21]:
from dotenv import load_dotenv
import os

# Load .env file from specific path
load_dotenv("C:/Users/minh1/local/lm-evaluation-harness/venv/.env")

hf_token = os.getenv('HF_TOKEN')

if hf_token:
    huggingface_hub.login(hf_token)
else:
    print("Huggingface token not found. Please login manually.")
    !huggingface-cli login

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Automated login for WandB via Colab Secrets. If you don't have this, it'll just prompt you later if you use wandb_args.

In [24]:
# Install all necessary dependencies
!pip install --upgrade pip setuptools wheel
!pip uninstall wandb -y
!pip install wandb --no-deps
!pip install docker-pycreds pathtools promise protobuf psutil requests sentry-sdk setproctitle typing-extensions

Found existing installation: wandb 0.19.3
Uninstalling wandb-0.19.3:
  Successfully uninstalled wandb-0.19.3
  Using cached wandb-0.19.3-py3-none-win_amd64.whl.metadata (10 kB)
Using cached wandb-0.19.3-py3-none-win_amd64.whl (19.7 MB)


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
          exec(compile('''
          ~~~~^^^^^^^^^^^^
          # This is <pip-setuptools-caller> -- a caller that pip uses to run setup.py
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          ...<31 lines>...
          exec(compile(setup_py_code, filename, "exec"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          ''' % ('C:\\Users\\minh1\\AppData\\Local\\Temp\\pip-install-nrffh5id\\pathtools_e81954648e7d4e0c9f65f98af3115f7c\\setup.py',), "<pip-setuptools-caller>", "exec"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<pip-setuptools-caller>", line 34, in <module>


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [25]:
# First install setuptools which includes distutils
!pip install setuptools
!pip install wandb

# Now try the WandB login
wandb_token = os.getenv('WANDB_API_KEY')

if wandb_token:
    os.environ["WANDB_API_KEY"] = wandb_token
    import wandb
    wandb.login()
else:
    print("WandB token not found. Continuing without logging into WandB.")

ModuleNotFoundError: No module named 'distutils'

Automated login for Goodfire API via Colab Secrets.

In [26]:
# Get GOODFIRE_API_KEY from environment variables (loaded from .env)
GOODFIRE_API_KEY = os.getenv('GOODFIRE_API_KEY')
if not GOODFIRE_API_KEY:
    raise ValueError("Please set GOODFIRE_API_KEY in your .env file")

#Step 2. Run evaluation

In [29]:
# Install PyTorch for Windows
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [30]:
import os

# Set environment variable for Goodfire API key if not already set
if 'GOODFIRE_API_KEY' not in os.environ:
    os.environ['GOODFIRE_API_KEY'] = GOODFIRE_API_KEY

# Run the evaluation using the command-line interface
# Note: Using sys.executable ensures we use the correct Python interpreter
import sys
!{sys.executable} -m lm_eval \
    --model goodfire \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B-Instruct \
    --tasks gsm8k_cot_llama \
    --num_fewshot 8 \
    --limit 10 \
    --batch_size auto \
    --log_samples \
    --output_path ./lm-eval-output/ \
    --gen_kwargs top_p=0.9,temperature=1.0,do_sample=True

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "c:\Users\minh1\local\lm-evaluation-harness\lm_eval\__init__.py", line 1, in <module>
    from .evaluator import evaluate, simple_evaluate
  File "c:\Users\minh1\local\lm-evaluation-harness\lm_eval\evaluator.py", line 10, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


#Reference: EleutherAI Eval Harness task list
For those curious to run other evals! Please note that Min P is currently only accessible for `generate_until` tasks. There is currently no easy way to index these tasks, I just Ctrl + F'd `generate_until` on the [EleutherAI Evals Harness Repo](https://github.com/EleutherAI/lm-evaluation-harness).

In [6]:
# Test Goodfire Client
import goodfire

client = goodfire.Client(api_key=GOODFIRE_API_KEY)

# Simple test call
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Say hello!"}],
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_completion_tokens=10
)

# Access response using ChatCompletion object attributes
print("Test response:", response.choices[0].message['content'])

Test response: Hello! It's nice to meet you. How


In [ ]:
 !lm-eval --tasks list